# Compute the atom centered Coulomb Matrix representation

To install QML library and compare rascal results with it
+ pip install qml ase

To install rascal:
+ mkdir ../build 
+ cd build
+ cmake -DCMAKE_BUILD_TYPE=Release ..
+ make install -j 4

In [1]:
%env OMP_NUM_THREADS=1

env: OMP_NUM_THREADS=1


In [2]:
%matplotlib inline
from matplotlib import pyplot as plt

In [35]:
import qml
from ase.io import read
from ase.visualize import view
import ase
import numpy as np
import sys, os
from copy import copy
from ase.build import make_supercell

In [6]:
import sys, os
sys.path.insert(0,'../build/')
import rascal as rc
from rascal.representations import SortedCoulombMatrix

In [7]:
# load a small subset of structures from QM9
frames = read('../tests/reference_data/dft-smiles_500.xyz',':')
cutoff = 3.
rep = SortedCoulombMatrix(cutoff,sorting_algorithm='row_norm')

In [8]:
new_frames = []
aa = []
for frame in frames:
    new_frames.append(make_supercell(frame,np.eye(3)*1))
print(np.max(list(map(len,new_frames))))

27


In [ ]:
# have a look at them
view(frames)

In [ ]:
# Compute the sorted coulomb matrices for the list of structures
%timeit -n 3 -r 3 features = rep.transform(new_frames)

In [9]:
# extract the feature matrix to compare with 
features = rep.transform(new_frames)
test = features.get_feature_matrix().T

In [13]:
!pwd

/local/git/librascal/examples


In [12]:
import ubjson as json

In [36]:
def load_json(fn):
    import json as js
    with open(fn,'r') as f:
        data = js.load(f)
    return data[str(data['ids'][0])]
def json2ase(f):
    return ase.Atoms(**{v:f[k] for k,v in
dict(positions='positions',atom_types='numbers',pbc='pbc',cell='cell').items()
})

In [37]:
with open('../tests/reference_data/sorted_coulomb_reference.ubjson','rb') as f:
    data_ref = json.load(f)
fns = data_ref['filenames'] 
cutoffs = data_ref['cutoffs']
sorts = ['row_norm','distance']


In [39]:
data = dict(rep_info=[])
for fn in fns:
    fn_r = '../tests/'+fn
    for cutoff in cutoffs:
        print(fn_r,cutoff)
        data['rep_info'].append([])
        for sort in sorts:
            rep = SortedCoulombMatrix(cutoff,sorting_algorithm=sort)
            frame = [json2ase(load_json(fn_r))]
            features = rep.transform(frame)
            test = features.get_feature_matrix()
            data['rep_info'][-1].append(dict(feature_matrix=test))

../tests/reference_data/CaCrP2O7_mvc-11955_symmetrized.json 2
../tests/reference_data/CaCrP2O7_mvc-11955_symmetrized.json 3
../tests/reference_data/CaCrP2O7_mvc-11955_symmetrized.json 4
../tests/reference_data/CaCrP2O7_mvc-11955_symmetrized.json 5
../tests/reference_data/small_molecule.json 2
../tests/reference_data/small_molecule.json 3
../tests/reference_data/small_molecule.json 4
../tests/reference_data/small_molecule.json 5


In [41]:
for a_r,a_t in zip(data_ref['rep_info'],data['rep_info']):
    for b_r,b_t in zip(a_r,a_t):
        f_r = b_r['feature_matrix']
        f_t = b_t['feature_matrix']

        print(np.allclose(f_r,f_t))

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [10]:
# To get the coulomb matrices from QML
def get_coulomb_ref(frame,size,cutoff,flavour):
    from qml.representations import generate_atomic_coulomb_matrix
    
    nuclear_charges = frame.get_atomic_numbers()
    coordinates = frame.get_positions()
    
    
    cm = generate_atomic_coulomb_matrix(nuclear_charges, coordinates, size = size, sorting = flavour,
    central_cutoff = cutoff, central_decay = -1, interaction_cutoff = 1e6, interaction_decay = -1,indices = None)
    
    return cm
def get_coulomb_refs(frames,size,cutoff,flavour='distance'):  
    cms = []
    for frame in frames:
        cms.append(get_coulomb_ref(frame,size,cutoff,flavour))
    return np.vstack(cms)

In [11]:
# Compare with reference
%timeit -n 3 -r 3 ref = get_coulomb_refs(new_frames,rep.size,cutoff,flavour='row-norm')
ref = get_coulomb_refs(new_frames,rep.size,cutoff,flavour='row-norm')
np.allclose(test,ref)

24 ms ± 1.01 ms per loop (mean ± std. dev. of 3 runs, 3 loops each)


True

In [ ]:
from rascal.utils import fps

In [ ]:
result = fps(test,n_select=500,starting_index=None,method='simple')

In [ ]:
plt.plot(result['fps_minmax_d2'])